In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
dataset = pd.read_csv('GOOGL.csv', index_col='Date', parse_dates=['Date'])

In [ ]:
# Split the dataset into training and testing sets
train_set = dataset[:'2018'].iloc[:, 1:2].values
test_set = dataset['2019':].iloc[:, 1:2].values

In [ ]:
# Normalize the dataset
sc = MinMaxScaler(feature_range=(0, 1))
train_set_scaled = sc.fit_transform(train_set)

In [ ]:
# Create a function to generate a sliding window of data
def create_dataset(dataset, look_back=60):
    X, Y = [], []
    for i in range(look_back, len(dataset)):
        X.append(dataset[i-look_back:i, 0])
        Y.append(dataset[i, 0])
    X, Y = np.array(X), np.array(Y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, Y

In [ ]:
# Generate the training and testing datasets
look_back = 60
X_train, Y_train = create_dataset(train_set_scaled, look_back)
X_test, Y_test = create_dataset(test_set, look_back)


In [ ]:
# Build the RNN model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the RNN model
model.fit(X_train, Y_train, epochs=100, batch_size=32)

Epoch 1/100
112/112 [==============================] - 14s 71ms/step - loss: 0.0091
Epoch 2/100
112/112 [==============================] - 10s 92ms/step - loss: 2.3923e-04
Epoch 3/100
112/112 [==============================] - 9s 79ms/step - loss: 2.2965e-04
Epoch 4/100
112/112 [==============================] - 11s 94ms/step - loss: 2.3552e-04
Epoch 5/100
112/112 [==============================] - 12s 111ms/step - loss: 2.3164e-04
Epoch 6/100
112/112 [==============================] - 7s 59ms/step - loss: 2.3587e-04
Epoch 7/100
112/112 [==============================] - 8s 68ms/step - loss: 2.2504e-04
Epoch 8/100
112/112 [==============================] - 8s 70ms/step - loss: 2.2990e-04
Epoch 9/100
112/112 [==============================] - 6s 57ms/step - loss: 1.8855e-04
Epoch 10/100
112/112 [==============================] - 8s 69ms/step - loss: 2.0127e-04
Epoch 11/100
112/112 [==============================] - 7s 60ms/step - loss: 2.3821e-04
Epoch 12/100
112/112 [==================

In [ ]:
# Make predictions on future Google stock prices
last_60_days = train_set[-60:]
last_60_days_scaled = sc.transform(last_60_days)
X_pred = np.array([last_60_days_scaled])
X_pred = np.reshape(X_pred, (X_pred.shape[0], X_pred.shape[1], 1))
pred_price = model.predict(X_pred)
pred_price = sc.inverse_transform(pred_price)
print(pred_price)

1/1 [==============================] - 1s 865ms/step
[[1071.5663]]
